In [1]:
import requests
import numpy as np
import pandas as pd
%run ../employee/links.ipynb

In [2]:
# req = requests.get(links_ns.taskrole.task+"4").json() # Risk of empty list
req = requests.get(links_ns.task.project+"4").json() # Risk of empty list
task = pd.json_normalize(req)


In [3]:
task

,taskId,code,name,type,startDate,endDate,status,progress,budget,comment,...,project.code,project.name,project.type,project.startDate,project.endDate,project.status,project.progress,project.budget,project.comment,project.project
0,8,A-03-23,Prototypage de l'application,ACTIVITY,2023-01-03,2023-02-22,FINISHED,None,None,DSESP,...,A-01-23,Application de suivi-évaluation des activités ...,PROJECT,2023-01-02,2023-05-31,IN_PROGRESS,52.0,700000,DSESP,None
1,9,A-04-23,Design des bases de données de l'application,ACTIVITY,2023-03-20,2023-03-24,IN_PROGRESS,None,None,DSESP,...,A-01-23,Application de suivi-évaluation des activités ...,PROJECT,2023-01-02,2023-05-31,IN_PROGRESS,52.0,700000,DSESP,None
2,10,A-04-23,Mise en production,ACTIVITY,2023-04-07,2023-04-15,SCHEDULED,None,None,DSESP,...,A-01-23,Application de suivi-évaluation des activités ...,PROJECT,2023-01-02,2023-05-31,IN_PROGRESS,52.0,700000,DSESP,None


### 1.Task bio data

In [4]:
# Request list of project tasks
req = requests.get(links_ns.task.single+"4").json() # Risk of empty list or 404
task = pd.json_normalize(req)
task.to_dict(orient="records")

[{'taskId': 4,
  'code': 'A-01-23',
  'name': "Application de suivi-évaluation des activités de l'ARTI",
  'type': 'PROJECT',
  'startDate': '2023-01-02',
  'endDate': '2023-05-31',
  'status': 'IN_PROGRESS',
  'progress': 52.0,
  'budget': 700000,
  'comment': 'DSESP',
  'project': None}]

### 2.Project Stats

In [9]:
data = dict()
data_task = dict()
data_resources = dict()

id = 4

req = requests.get(links_ns.task.project+str(id))
if req.status_code == 200 and len(req.json())>0:
    task = pd.json_normalize(req.json())
    data_task = task
    nb_task = task.shape[0] # First value
    # For each task compute number of employees involved
    nb_emp = list([])
    role_tmp = pd.DataFrame()
    for taskId in task.taskId:
        req = requests.get(links_ns.taskrole.task+str(taskId)).json() # Risk of empty list
        if len(req)>0:
            role = pd.json_normalize(req)
            role_tmp = pd.concat([role_tmp, role])
            nb_emp.append(role["position.positionId"].values)
    data_resources = role_tmp
    if len(nb_emp)>0:
        nb_emp = len(set(np.concatenate(nb_emp))) # Second value
    else: nb_emp=0
    # Project budget
    req = requests.get(links_ns.task.single+str(id)).json() # Risk of empty list
    task = pd.json_normalize(req)
    budget = task.budget.max()
    data = {"tasks":nb_task,"resources":nb_emp,"budget":budget}
    # print(nb_task, nb_emp, budget)
else: # retuen empty dict
    data = {"tasks":0,"resources":0,"budget":0}
data

{'tasks': 3, 'resources': 1, 'budget': 700000}

### 3. Activity List

In [15]:
pd.DataFrame([data])

,tasks,resources,budget
0,3,1,700000


In [16]:
tasks_df = data_task[["taskId","code","name","type","status","endDate"]]
resources_df = data_resources[["role","task.taskId","position.employee.employeeId",
                               "position.employee.firstName","position.employee.lastName"]]

df = pd.merge(left=tasks_df, right=resources_df, left_on="taskId", right_on="task.taskId")
df.drop(columns=["task.taskId"], inplace=True)
rename_col = ["taskId","code","name","type","status","endDate","role","employeeId","firstName","lastName"]
df.columns = rename_col

df

,taskId,code,name,type,status,endDate,role,employeeId,firstName,lastName
0,8,A-03-23,Prototypage de l'application,ACTIVITY,FINISHED,2023-02-22,RESPONSABLE,6,Alassane,CISSE
1,9,A-04-23,Design des bases de données de l'application,ACTIVITY,IN_PROGRESS,2023-03-24,RESPONSABLE,6,Alassane,CISSE
2,10,A-04-23,Mise en production,ACTIVITY,SCHEDULED,2023-04-15,RESPONSABLE,6,Alassane,CISSE


### 1.4 Resources list

In [22]:
emp_task_list = data_resources[["task.taskId","position.employee.employeeId","position.employee.lastName","position.employee.firstName","role","task.name","task.status"]]
rename_col = ["taskId","employeeId","lastName","firstName","role","taskName","status"]
emp_task_list.columns = rename_col
#emp_task_list

In [23]:
emp_task_list

,taskId,employeeId,lastName,firstName,role,taskName,status
0,8,6,CISSE,Alassane,RESPONSABLE,Prototypage de l'application,FINISHED
0,9,6,CISSE,Alassane,RESPONSABLE,Design des bases de données de l'application,IN_PROGRESS
0,10,6,CISSE,Alassane,RESPONSABLE,Mise en production,SCHEDULED


In [24]:
# compute total number of tasks per employee
total_tasks = emp_task_list.groupby('employeeId')['taskId'].count()
# compute number of finished tasks per employee
finished_tasks = emp_task_list[emp_task_list['status'] == 'FINISHED'].groupby('employeeId')['taskId'].count()
# compute ratio of finished tasks per employee
ratio_finished = finished_tasks / total_tasks
# merge the three results into a single DataFrame
result = pd.concat([total_tasks, finished_tasks, ratio_finished], axis=1)
result.columns = ['total_tasks', 'finished_tasks', 'ratio_finished']
result.reset_index(inplace=True)

pd.merge(left=result, right=emp_task_list.groupby("employeeId")[["lastName","firstName","role"]].first().reset_index(), how='left')

,employeeId,total_tasks,finished_tasks,ratio_finished,lastName,firstName,role
0,6,3,1,0.333333,CISSE,Alassane,RESPONSABLE
